In [97]:
class Parser:
    def __init__(self, filename):
        with open(filename, 'r') as f:
            self.lines = f.readlines()
        self.index = 0
        self.commands = [line.strip().replace(' ', '') for line in self.lines if not line.startswith('//') and line.strip() != '']
        self.current_line = None

    def has_more_commands(self):
        return self.index < len(self.commands)
    
    def advance(self):
        if self.has_more_commands():
            self.current_line = self.commands[self.index]
            self.index += 1

    def command_type(self):
        if self.current_line.startswith('@'):
            return 'A_COMMAND'
        if any(char in self.current_line for char in ['=', ';']):
            return 'C_COMMAND'
        else:
            return 'L_COMMAND'
        
    def symbol(self):
        if self.command_type() == 'A_COMMAND':
            return self.current_line.strip('@')
        elif self.command_type() == 'L_COMMAND':
            return self.current_line.strip('()')
        
    def dest(self):
        if self.command_type() == 'C_COMMAND':
            if '=' in self.current_line:
                return self.current_line.split('=')[0]
            else:
                return None
            
    def comp(self):
        if self.command_type() == 'C_COMMAND':
            if ';' and '=' in self.current_line:
                return self.current_line.split('=')[1].split(';')[0]
            elif '=' in self.current_line:
                return self.current_line.split('=')
            elif ';' in self.current_line:
                return self.current_line.split(';')
            else:
                return self.current_line
            
    def jump(self):
        if self.command_type() == 'C_COMMAND':
            if ';' in self.current_line:
                return self.current_line.split(';')[1]
            else:
                return None

In [86]:
class Code:
    def __init__(self):
        self.dest_map = {
            None: '000',
            'M': '001',
            'D': '010',
            'MD': '011',
            'A': '100',
            'AM': '101',
            'AD': '110',
            'AMD': '111'
        }

        self.comp_map = {
            '0': '0101010',
            '1': '0111111',
            '-1': '0111010',
            'D': '0001100',
            'A': '0110000',
            'M': '1110000',
            '!D': '0001101',
            '!A': '0110001',
            '!M': '1110001',
            '-D': '0001111',
            '-A': '0110011',
            '-M': '1110011',
            'D+1': '0011111',
            'A+1': '0110111',
            'M+1': '1110111',
            'D-1': '0001110',
            'A-1': '0110010',
            'M-1': '1110010',
            'D+A': '0000010',
            'D+M': '1000010',
            'D-A': '0010011',
            'D-M': '1010011',
            'A-D': '0000111',
            'M-D': '1000111',
            'D&A': '0000000',
            'D&M': '1000000',
            'D|A': '0010101',
            'D|M': '1010101'
        }

        self.jump_map = {
            None: '000',
            'JGT': '001',
            'JEQ': '010',
            'JGE': '011',
            'JLT': '100',
            'JNE': '101',
            'JLE': '110',
            'JMP': '111'
        }

    def dest(self, symbol):
        return self.dest_map[symbol]
    
    def comp(self, symbol):
        return self.comp_map[symbol]
    
    def jump(self, symbol):
        return self.jump_map[symbol]


In [127]:
with open('Add.hack', 'w') as file:
    parser = Parser('Add.asm')
    coder = Code()

    while parser.has_more_commands():
        parser.advance()

        if parser.command_type() in ['C_COMMAND']:
            dest_field = coder.dest(parser.dest())
            comp_field = coder.comp(parser.comp())
            jump_field = coder.jump(parser.jump())
            machine_instruction = f'111{comp_field}{dest_field}{jump_field}'

            print(machine_instruction)
            file.write(machine_instruction + '\n')

        elif parser.command_type() in ['A_COMMAND']:
            decimal_value = parser.symbol()
            binary_value = format(int(decimal_value), '015b')
            machine_instruction = f'0{binary_value}'    # 0 = opcode, 15-bit address

            print(machine_instruction)
            file.write(machine_instruction + '\n')

0000000000000010
1110110000010000
0000000000000011
1110000010010000
0000000000000000
1110001100001000
